# GRU - POS tagging

A POS é uma categoria gramatical de palavras que são usadas da mesma forma através de multiplex frases. Exemplo de POS tagging são nomes, verbos, adjetivos, etc. Por exemplo, nomes são tipicamente utilizados para identificar coisas, verbos são tipicamente utilizados para identificar o que esta fazendo, e adjetivos para descrever alguns atributos das coisas. POS tagging normalmente é realizado manualmente, mais atualmente é feito automaticamente utilizando modelos estatísticos. Nos últimos anos Deep Learning tem sido aplicado nestes problemas com bons resultados (para maior informação ler artigo: [Natural Language Processing (almost) from Scratch](http://www.jmlr.org/papers/volume12/collobert11a/collobert11a.pdf) , R. Collobert, Journal of Machine Learning Research. 2011).

Para nossos dados de treinamento, precisamos de frases marcadas com parte das tags da fala. O dataset [The Peen Treebank](https://catalog.ldc.upenn.edu/ldc99t42), é uma anotação de 4.5 milhões de palavras de Inglês Americano. A data não é livre, um 10% do dataset é disponível em NLTK (http://www.nltk.org/), que utilizaremos em nossa rede.

O modelo pegara uma seqüência de palavras em uma frase e predicará o correspondente POS tags para cada palavra. Assim, para uma seqüência contida pelas palavras [The, cat, sat, on, the, mat.], a seqüência de saída é [DT, NN, VB, IN, DT, NN]. Ref: [https://cs.nyu.edu/grishman/jet/guide/PennPOS.html](https://cs.nyu.edu/grishman/jet/guide/PennPOS.html).

In [1]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras_tqdm import TQDMNotebookCallback
from sklearn.model_selection import train_test_split
import collections
import nltk
import numpy as np
import os

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla C2070 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN not available)


# Pre-processing

## 1. Dataset

Baixamos os dados desde o NLTK num formato. Utilizamos o seguinte código em Python para baixar os dados em 2 arquivos paralelos, um para as palavras nas orações, e outro para os POS tags.

In [2]:
DATA_DIR="data"

fedata = open(os.path.join(DATA_DIR, "treebank_sents.txt"),"w")
ffdata = open(os.path.join(DATA_DIR, "treebank_poss.txt"), "w")

sents = nltk.corpus.treebank.tagged_sents()
for sent in sents:
    words, poss = [], []
    for word,pos in sent:
        if pos == "-NONE-":
            continue
        words.append(word)
        poss.append(pos)
    fedata.write("{}\n".format(" ".join(words)))
    ffdata.write("{}\n".format(" ".join(poss)))
    
fedata.close()
ffdata.close()

## 2. Create a Word Counter

Queremos explorar os dados um pouco para encontrar o tamanho de vocabulário a utilizar. Neste caso, temos que considerar 2 tipos diferentes de vocabulários, a fonte de vocabulários para as palavras e para os tags. Precisamos encontrar um número único de palavras em cada vocabulário, também precisamos encontrar o número máximo de palavras em uma frase em nosso dataset de treinamento e o número de instancias.

In [3]:
def parse_sentences(filename):
    word_freqs = collections.Counter()
    num_recs, maxlen = 0, 0
    fin = open(filename, 'rb')
    for line in fin:
        words = line.strip().lower().split()
        for word in words:
            word_freqs[word] += 1
        if len(words) > maxlen:
            maxlen = len(words)
        num_recs += 1
    fin.close()
    return word_freqs, maxlen, num_recs

s_wordfreqs, s_maxlen, s_numrecs = parse_sentences(os.path.join(DATA_DIR, "treebank_sents.txt"))
t_wordfreqs, t_maxlen, t_numrecs = parse_sentences(os.path.join(DATA_DIR, "treebank_poss.txt"))
print(len(s_wordfreqs), s_maxlen, s_numrecs, len(t_wordfreqs), t_maxlen, t_numrecs)

10947 249 3914 45 249 3914


Número de palavras únicas: **10947** <br/>
Número de POS tags únicos: **45** <br/>
Número máximo de palavras por frase : **249**<br/>
Número de frases (10%) : **3914**

## 3. Create a Lookup Table

Cada entrada é representada por uma sequencia de índices das palavras. A saída é a sequencia de índices dos POS tags. Então, precisamos construir **lookup table** para transformar entre as palavras/POS tags e seu índice correspondente.

In [31]:
MAX_SEQLEN = 250
S_MAX_FEATURES = 5000
T_MAX_FEATURES = 45

In [32]:
s_vocabsize = min(len(s_wordfreqs), S_MAX_FEATURES) + 2
s_word2index = {x[0]: i+2 for i, x in enumerate(s_wordfreqs.most_common(S_MAX_FEATURES))}
s_word2index["PAD"] = 0
s_word2index["UNK"] = 1
s_index2word = {v:k for k, v in s_word2index.items()}

t_vocabsize = len(t_wordfreqs) + 1
t_word2index = {x[0]:i for i, x in enumerate(t_wordfreqs.most_common(T_MAX_FEATURES))}
t_word2index["PAD"] = 0
t_index2word = {v:k for k, v in t_word2index.items()}

## 4. Create input and output dataset for train and test

O seguinte passo é construir nosso dataset para a nossa rede. Utilizaremos os **Lookup table** para converter as frases em ID de sequencias com dimensão MAX_SEQLEN (250). Os rótulos precisam ser estruturadas como uma sequencia de **one-hot** vectors de dimensão T_MAX_FEATURES + 1 (46), também de dimensão MAX_SEQLEN (250). A função ```build_tensor``` lê os dados dos dois arquivos e converte para os tensores de entradas e saídas.

In [6]:
def build_tensor(filename, numrecs, word2index, maxlen, make_categorical=False, num_classes=0):
    data = np.empty((numrecs,), dtype=list)
    fin = open(filename, "rb")
    i = 0
    for line in fin:
        wids = []
        for word in line.strip().lower().split():
            if word in word2index:
                wids.append(word2index[word])
            else:
                wids.append(word2index["UNK"])
        if make_categorical:
            data[i] = np_utils.to_categorical(wids, num_classes=num_classes) #[[[01000],[10000],[0001]]
        else:
            data[i] = wids #[[8,16,1]]
        i += 1
    fin.close()
    pdata = sequence.pad_sequences(data, maxlen=maxlen)
    return pdata

X = build_tensor(os.path.join(DATA_DIR, "treebank_sents.txt"), s_numrecs, s_word2index, MAX_SEQLEN)
Y = build_tensor(os.path.join(DATA_DIR, "treebank_poss.txt"), t_numrecs, t_word2index, MAX_SEQLEN, 
                 True, t_vocabsize)

Finalmente, dividimos nossos dados de treinamento e teste (80-20):

In [7]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y, test_size=0.2, random_state=42)